<a href="https://colab.research.google.com/github/JoeHeeJae/Movie-Recommendation-System-Colon-coding/blob/main/Latent_factor_collaborative_filtering_Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Latent factor collaborative filtering Matrix Factorization
행렬 분해를 이용한 추천 시스템 

In [47]:
import pandas as pd
import numpy as np
import sklearn

##데이터 import

In [48]:
df_ratings = pd.read_csv('/content/drive/MyDrive/Data_csv/ratings.csv')
df_movies = pd.read_csv('/content/drive/MyDrive/Data_csv/movies.csv')

##데이터 전처리

###사용자-영화 평점 데이터를 pivot table 형식으로 변환
(사용자 별 각 영화 평점) 

In [49]:
df_user_movie_ratings = df_ratings.pivot(
    index = 'userId',
    columns = 'movieId',
    values = 'rating'
).fillna(0)

In [50]:
df_user_movie_ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
from numpy import ndarray

In [52]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것.
matrix = df_user_movie_ratings.values

# user_rating_mean은 사용자의 평균 평점
user_rating_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_rating_mean.reshape(-1, 1)

In [53]:
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [54]:
# 원래 0~5까지 존재하던 평점 데이터 분포가 각 사용자의 평점 평균을 배니까 0.00025 이런식으로 변경됨.
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
1,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,3.970770,...,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230
2,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,...,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075
3,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,3.902162,...,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838
4,-0.043128,-0.043128,3.956872,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,...,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128


###SVD(Singular Value Decomposition)
특이값 분해 사용

In [55]:
# scipy의 SVD 사용
!pip install scipy
from scipy.sparse.linalg import svds
# U 행렬, sigma 행렬, V 천치 행렬을 반환.
U, sigma, Vt = svds(matrix_user_mean, k = 12)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(671, 12)
(12,)
(12, 9066)


In [57]:
# 현재 sigma 값은 행렬 0이 아닌 값만 1차원 행렬로 표현된 상태.
# 0이 포함된 대칭행렬로 변환이 필요 > np.diag 사용 

In [58]:
sigma = np.diag(sigma)

In [59]:
sigma.shape

(12, 12)

In [60]:
sigma[0]

array([105.72437051,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [61]:
sigma[1]

array([  0.        , 110.15522471,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

즉, 
1. 원본 user-movie 평점 행렬이 있었음
2. 이를 user의 평균 점수를 빼서 matrix_user_mean 행렬로 변환
3. 2번의 값을 SVD를 적용해 U, Sigma, Vt 행렬로 변환
4. 위 Sigma 행렬은 0이 포함되지 않은 값으로 구성되어있음. 이를 대칭행렬로 변환

In [62]:
# SVD 적용하여 분해한 matrix_user_mean을 원본 행렬로 복구
# U, Sigma, Vt 의 내적을 수행 > np.dot(np.dot(U, sigma),Vt)
# + 사용자 평균 rating 적용
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [63]:
# SVD 특이값 분해를 사용해 행렬 분해 기반으로 데이터를 변경
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns= df_user_movie_ratings.columns)
df_svd_preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.079686,0.021779,-0.013837,-0.005870,-0.028877,0.032371,0.000715,-0.004428,-0.005219,0.038195,...,-0.004324,-0.004352,0.010478,-0.004256,-0.003944,-0.005674,0.018157,-0.005575,-0.005297,-0.003766
1,1.428452,1.608841,0.529476,0.168278,0.520809,1.107473,0.529719,0.089376,0.296270,1.970031,...,0.013227,-0.002275,0.020680,-0.005245,-0.007644,-0.021019,0.031243,-0.000957,-0.000753,0.026901
2,0.977246,0.396971,0.000299,0.027444,0.021287,0.141458,-0.057134,0.031633,-0.012538,0.383576,...,0.002761,0.004907,-0.014190,-0.000251,-0.006007,-0.003189,-0.026916,0.014637,0.013287,-0.005741
3,1.870844,1.169993,0.252202,0.094831,-0.181713,-0.511953,-0.027820,-0.143080,0.013247,1.461694,...,0.026412,-0.027245,0.054681,0.018450,0.034544,-0.035740,0.088889,-0.019365,-0.017113,0.066559
4,1.182777,0.924903,0.075998,0.061505,0.602680,-0.159825,0.339925,0.081534,-0.079666,0.535018,...,-0.029124,-0.029357,0.009064,-0.029092,-0.030890,-0.057453,0.026344,-0.024027,-0.024614,-0.032752


##파이썬 함수 만들기
- 인자로 사용자 아이디, 영화 정보 테이블, 평점 테이블 등을 받음
- 사용자 아이디에 SVD로 나온 결과의 영화 평점이 가장 높은 데이터 순으로 정렬
- 사용자가 본 데이터를 제외
- 사용자가 안 본 영화에서 평점이 높은 것을 추천

In [74]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
  #현재는 index로 적용이 되어 있으며로 user_id - 1을 해야함
  user_row_number = user_id - 1

  #최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 > 영화 평점이 높은 순으로 정렬
  sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

  # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다.
  user_data = ori_ratings_df[ori_ratings_df.userId == user_id]

  # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다.
  user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending=False)

  # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
  recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]

  # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다.
  recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')

  # 컬럼 이름 바꾸고 정렬해서 return
  recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False)

  return user_history, recommendations


## 결과


In [75]:
already_rated, predictions = recommend_movies(df_svd_preds, 330, df_movies, df_ratings, 10)

In [77]:
already_rated.head(10)

,userId,movieId,rating,timestamp,title,genres
47,330,1094,5.0,948577200,"Crying Game, The (1992)",Drama|Romance|Thriller
14,330,213,5.0,948577039,Burnt by the Sun (Utomlyonnye solntsem) (1994),Drama
34,330,527,5.0,948577164,Schindler's List (1993),Drama|War
45,330,1035,5.0,948574126,"Sound of Music, The (1965)",Musical|Romance
48,330,1172,5.0,948575236,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
53,330,1249,5.0,948576522,"Femme Nikita, La (Nikita) (1990)",Action|Crime|Romance|Thriller
27,330,373,5.0,948576522,Red Rock West (1992),Thriller
58,330,1428,5.0,948731410,Angel Baby (1995),Drama
61,330,1575,5.0,948577442,Gabbeh (1996),Drama
67,330,1683,5.0,948578017,"Wings of the Dove, The (1997)",Drama|Romance


In [78]:
already_rated, predictions = recommend_movies(df_svd_preds, 100, df_movies, df_ratings, 10)

In [79]:
already_rated.head(10)


,userId,movieId,rating,timestamp,title,genres
23,100,1073,5.0,854194056,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical
5,100,32,5.0,854193977,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
0,100,1,4.0,854193977,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,100,3,4.0,854194024,Grumpier Old Men (1995),Comedy|Romance
22,100,802,4.0,854194111,Phenomenon (1996),Drama|Romance
19,100,745,4.0,854194208,Wallace & Gromit: A Close Shave (1995),Animation|Children|Comedy
13,100,608,4.0,854194024,Fargo (1996),Comedy|Crime|Drama|Thriller
24,100,1356,4.0,854194086,Star Trek: First Contact (1996),Action|Adventure|Sci-Fi|Thriller
4,100,25,4.0,854193977,Leaving Las Vegas (1995),Drama|Romance
8,100,86,3.0,854194208,White Squall (1996),Action|Adventure|Drama
